# Pandas Dataframe Agent

This notebook shows how to use agents to interact with a pandas dataframe. It is mostly optimized for question answering.

**NOTE: this agent calls the Python agent under the hood, which executes LLM generated Python code - this can be bad if the LLM generated Python code is harmful. Use cautiously.**

In [1]:
import json
filepath = 'credentials.json'
file = open(filepath, 'r')

# Open the credentials file with json.load
credentials = json.load(file)
file.close()

In [103]:
!pip install langchain
!pip install pandas
!pip install openpyxl
!pip install openai
!pip install tabulate

In [2]:
import pandas as pd

In [3]:
from langchain.agents import create_pandas_dataframe_agent

In [6]:
def load_excel_data(file_path, sheet_name):
    """
    This function loads data from a specific sheet of an Excel file using pandas.
    
    Parameters:
    file_path (str): The file path of the Excel file
    sheet_name (str): The name of the sheet to be loaded
    
    Returns:
    pandas.DataFrame: The data from the specified sheet
    """
    try:
        # Load the Excel file
        excel_data = pd.ExcelFile(file_path)
        
        # Check if the sheet exists
        if sheet_name not in excel_data.sheet_names:
            raise ValueError(f"Sheet '{sheet_name}' does not exist in the Excel file")
        
        # Load the specified sheet
        sheet_data = pd.read_excel(excel_data, sheet_name=sheet_name)
        
        return sheet_data
    except Exception as e:
        # Log the error
        print(f"Error: {e}")
        return None

In [7]:
from langchain.llms import OpenAI

df = load_excel_data('Evaluate_US Orphan Ph23.xlsx', 'Clean')

In [8]:
df

,Source:,Evaluate Ltd,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21
0,Report:,US Orphan Ph2/3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Report Date,2023-05-10 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,All Financial Data in US $ (mln),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Annual Sales USA - Sales,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Company,Product,Generic Name,Lead Asset,First Global Launch (WW),Patent Expiry,Therapeutic Category,Therapeutic Subcategory,Mechanism of Action,USA Phase (Current),...,2023,2024.0,2025.0,2026.0,2027.0,2028.0,Disease Biomarker,Biological/Chemical,Therapy Type,Market Cap
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
569,NaN,R&D,Phase III,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
570,NaN,R&D,Filed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
571,Orphan Drug Status (USA OD),2 Items,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
572,NaN,Designated,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
df.drop(df.index[0:4], axis=0, inplace=True)

In [10]:
df.columns = df.iloc[0]

In [11]:
df = df.rename(columns=df.iloc[0]).drop(df.index[0]).reset_index(drop=True)

In [12]:
df.columns

Index([                    'Company',                     'Product',
                      'Generic Name',                  'Lead Asset',
          'First Global Launch (WW)',               'Patent Expiry',
              'Therapeutic Category',     'Therapeutic Subcategory',
               'Mechanism of Action',         'USA Phase (Current)',
       'Orphan Drug Status (USA OD)',          'Indication Level 3',
                              '2023',                        2024.0,
                              2025.0,                        2026.0,
                              2027.0,                        2028.0,
                 'Disease Biomarker',         'Biological/Chemical',
                      'Therapy Type',                  'Market Cap'],
      dtype='object', name=4)

In [154]:
llm = OpenAI(temperature=0, openai_api_key=credentials['OPENAI_API_KEY'])
agent = create_pandas_dataframe_agent(llm, df, verbose=True)

In [155]:
agent.run("which rows have the largest growth in values from 2023 to 2028.0?")

'The rows with the largest growth in values from 2023 to 2028.0 are rows 2, 6, 0, 42, and 7.'

In [19]:
output = agent.run("what Therapy Type has the largest market captilization and within that what row has the largest market cap?")



> Entering new AgentExecutor chain...
Thought: I need to find the Therapy Type with the largest market cap and then find the row with the largest market cap within that Therapy Type
Action: python_repl_ast
Action Input: df.groupby('Therapy Type')['Market Cap'].max()
Observation: Therapy Type
Combination therapy    142857.605384
Monotherapy            418526.122779
Unclassified           418526.122779
Name: Market Cap, dtype: object
Thought: I need to find the row with the largest market cap
Action: python_repl_ast
Action Input: df[df['Market Cap'] == df['Market Cap'].max()]
Observation: 4              Company      Product                Generic Name   
14   Johnson & Johnson  Nipocalimab                 nipocalimab  \
72   Johnson & Johnson     AAV-RPGR  botaretigene sparoparvovec   
95   Johnson & Johnson  Talquetamab                 talquetamab   
197  Johnson & Johnson       GemRIS   gemcitabine hydrochloride   
216  Johnson & Johnson    AAV-CNGB3    entacingene turiparvovec   
25

In [16]:
agent.run("which company is leading the development of new products with largest increase from 2023 to 2028.0?")



> Entering new AgentExecutor chain...
Thought: I need to find the company with the largest increase in product development from 2023 to 2028.0
Action: python_repl_ast
Action Input: df.groupby('Company')['2023', '2028.0'].sum().sort_values(by='2028.0', ascending=False)
Observation: ValueError('Cannot subset columns with a tuple with more than one element. Use a list instead.')
Thought: I need to use a list instead of a tuple
Action: python_repl_ast
Action Input: df.groupby('Company')[['2023', '2028.0']].sum().sort_values(by='2028.0', ascending=False)
Observation: KeyError("Columns not found: '2028.0'")
Thought: I need to make sure the column name is correct
Action: python_repl_ast
Action Input: df.groupby('Company')[['2023', '2027.0']].sum().sort_values(by='2027.0', ascending=False)
Observation: KeyError("Columns not found: '2027.0'")
Thought: I need to make sure the column name is correct
Action: python_repl_ast
Action Input: df.groupby('Company')[['2023', '2027.0']].sum().sort_value

KeyboardInterrupt: 

In [18]:
output = agent.run("Show all the types of drugs that are expect to see the largest growth from 2023?")
output



> Entering new AgentExecutor chain...
Thought: I need to find the drugs that have the highest growth from 2023
Action: python_repl_ast
Action Input: df.sort_values(by=['2023', '2024.0', '2025.0', '2026.0', '2027.0', '2028.0'], ascending=False).head()
Observation: KeyError('2024.0')
Thought:

KeyboardInterrupt: 

In [27]:
from langchain import PromptTemplate
from langchain.chains import LLMChain

In [30]:
template = """
You are a professional biotechnology consultant who is working on the next generation of patient therapies. Explain the analysis provided by this summary ({summary}) of the largest market cap biologic treatment. Explain it using advanced medical verbiage and in depth with 500 words. 
"""
prompt = PromptTemplate(
    input_variables=["summary"],
    template=template,
)

In [33]:
chain = LLMChain(llm=llm, prompt=prompt)
explanation =chain.run(output)

In [32]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=0,
)

In [34]:
texts = text_splitter.create_documents([explanation])

In [36]:
texts[0].page_content

"The analysis provided by this summary indicates that the therapy type with the largest market cap is unclassified, and the row with the largest market cap is Johnson & Johnson's Nipocalimab. This is an important insight into the biologic treatment market, as it provides a glimpse into the current trends in the industry."

In [41]:
from langchain.embeddings import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(openai_api_key=credentials['OPENAI_API_KEY'])

In [43]:
query_result = embeddings.embed_query(texts[0].page_content)
query_result

[-0.00016533053712919354,
 0.009933059103786945,
 0.001599572948180139,
 -0.01868896372616291,
 -0.009847086854279041,
 -0.007816827856004238,
 -0.021995574235916138,
 0.009179151616990566,
 -0.019032850861549377,
 -0.0018930346705019474,
 0.0018302090466022491,
 0.030526630580425262,
 -0.0187021903693676,
 -0.014892974868416786,
 0.004219235386699438,
 0.009000594727694988,
 0.028886551037430763,
 0.0001810369431041181,
 -0.0037034039851278067,
 0.011507005430758,
 -0.006527249701321125,
 0.03859476000070572,
 -0.021453291177749634,
 -0.008881556801497936,
 -0.023225633427500725,
 0.014681351371109486,
 0.0020963912829756737,
 -0.035235244780778885,
 -0.00933786854147911,
 0.009212217293679714,
 -0.005297190509736538,
 0.006365225650370121,
 -0.00962885096669197,
 0.005555105861276388,
 -0.014588766731321812,
 -0.004662320949137211,
 0.014522634446620941,
 0.006864523980766535,
 0.023979540914297104,
 -0.002418785821646452,
 0.01751181110739708,
 0.003428955329582095,
 0.0167049970477

In [46]:
import os, pinecone
from langchain.vectorstores import Pinecone

In [50]:
pinecone.init(api_key=credentials["PINECONE_API_KEY"],
              environment=credentials["PINECONE_ENV"])

In [52]:
index_name = "langchain-quickstart"
search = Pinecone.from_documents(texts, embeddings, index_name=index_name)

In [54]:
query = "What is magical about an Johnson and Johnson?"
result = search.similarity_search(query)
result

[Document(page_content="The analysis provided by this summary indicates that the therapy type with the largest market cap is unclassified, and the row with the largest market cap is Johnson & Johnson's Nipocalimab. This is an important insight into the biologic treatment market, as it provides a glimpse into the current trends in the industry.", metadata={}),
 Document(page_content="The market cap of a biologic treatment is a measure of its potential value. It is calculated by taking the total market value of the company's shares and dividing it by the number of shares outstanding. The higher the market cap, the more valuable the company's shares are.", metadata={}),
 Document(page_content='The analysis provided by this summary indicates that the therapy type with the largest market cap is unclassified. This means that the biologic treatment market is currently dominated by treatments that have not yet been classified into a specific therapeutic category. This could be due to the fact 

In [56]:
from langchain.agents.agent_toolkits import create_python_agent
from langchain.tools.python.tool import PythonREPLTool
from langchain.python import PythonREPL
from langchain.llms.openai import OpenAI

In [57]:
agent_executor = create_python_agent(
    llm = llm,
    tool = PythonREPLTool(),
    verbose=True
)

In [69]:
agent_executor.run("Calculate the tangent of a 180 degree angle")

'-1.2246467991473532e-16'